## With low-level RDD API (unstructured)

In [ ]:
#|output: false
from pyspark import SparkContext

sc = SparkContext('local')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/22 22:28:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
dataRDD = sc.parallelize([('Brooke', 20), ('Denny', 31), ('Jules', 30), ('TD', 35), ('Brooke', 25)])

In [ ]:
dataRDD.first()

('Brooke', 20)

In [ ]:
dataRDD.collect() # only use when the dataset is small

[('Brooke', 20), ('Denny', 31), ('Jules', 30), ('TD', 35), ('Brooke', 25)]

In [ ]:
dataRDD.take(2) # use `take(n)` to get the first n rows

[('Brooke', 20), ('Denny', 31)]

In [ ]:
mapedRDD = dataRDD.map(lambda x: (x[0], (x[1], 1)))
mapedRDD.collect()

[('Brooke', (20, 1)),
 ('Denny', (31, 1)),
 ('Jules', (30, 1)),
 ('TD', (35, 1)),
 ('Brooke', (25, 1))]

In [ ]:
reducedRDD = mapedRDD.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
reducedRDD.collect()

[('Brooke', (45, 2)), ('Denny', (31, 1)), ('Jules', (30, 1)), ('TD', (35, 1))]

In [ ]:
mapedRDD = reducedRDD.map(lambda x: (x[0], x[1][0]/x[1][1]))
mapedRDD.collect()

[('Brooke', 22.5), ('Denny', 31.0), ('Jules', 30.0), ('TD', 35.0)]

In [ ]:
#|output: false
sc.stop()

## With high-level DSL operates and DataFrame API (structured)

> DSL: domain specific language

In [ ]:
#|output: false
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg

spark = SparkSession.builder.appName('AvgAges').getOrCreate()

In [ ]:
data_df = spark.createDataFrame([('Brooke', 20), ('Denny', 31), ('Jules', 30), ('TD', 35), ('Brooke', 25)], 
                                ['name', 'age'])
data_df.show()

+------+---+
|  name|age|
+------+---+
|Brooke| 20|
| Denny| 31|
| Jules| 30|
|    TD| 35|
|Brooke| 25|
+------+---+



In [ ]:
data_df.show(1)

+------+---+
|  name|age|
+------+---+
|Brooke| 20|
+------+---+
only showing top 1 row



In [ ]:
data_df.take(2) # Get the first 2 rows

[Row(name='Brooke', age=20), Row(name='Denny', age=31)]

In [ ]:
data_df.tail(2) # Get the last 3 rows

[Row(name='TD', age=35), Row(name='Brooke', age=25)]

In [ ]:
avg_df = data_df.groupBy('name').agg(avg('age'))
avg_df.show()

+------+--------+
|  name|avg(age)|
+------+--------+
|Brooke|    22.5|
| Denny|    31.0|
| Jules|    30.0|
|    TD|    35.0|
+------+--------+



:::{.callout-note}
            
This version is far more expressive and simpler than the previous one.

:::

In [ ]:
#|output: false
spark.stop()